In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import linalg_ops
from tensorflow.python.ops import math_ops
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns

ImportError: No module named seaborn

In [7]:
@ops.RegisterGradient("gradient_no_unitary_adjustment")
def _test1(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan with no adjustment for subspace"""
    e = op.outputs[0]
    v = op.outputs[1]
    #dim = v.get_shape()
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_grassman = grad_v# - math_ops.batch_matmul(math_ops.batch_matmul(v, array_ops.transpose(grad_v)), v)
            grad_a = math_ops.batch_matmul(grad_grassman, math_ops.batch_matmul(E, array_ops.transpose(grad_v)))+math_ops.batch_matmul(grad_v, math_ops.batch_matmul(E, array_ops.transpose(grad_grassman)))
    return grad_a

In [8]:
@ops.RegisterGradient("grassman_with_2d")
def _test1(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan with no adjustment for subspace"""
    e = op.outputs[0]
    v = op.outputs[1]
    #dim = v.get_shape()
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            v_proj = array_ops.slice(v, [0,0], [20,2])
            grad_grassman = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v_proj, array_ops.transpose(v_proj)), grad_v)
            grad_a = math_ops.batch_matmul(grad_grassman, math_ops.batch_matmul(E, array_ops.transpose(grad_v)))+math_ops.batch_matmul(grad_v, math_ops.batch_matmul(E, array_ops.transpose(grad_grassman)))
    return grad_a

In [9]:
def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=0.5, p_out=0.1, seed=None):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out, seed=seed)
    A = nx.adjacency_matrix(G).todense()
    
    return A

In [10]:
def target_subspace(adj, groupsize, communities, diag, dim_proj):
    normalizer = tf.cast(2.0*groupsize*communities, dtype=tf.float64)
    total_degree = tf.cast(tf.reduce_sum(adj), dtype=tf.float64)
    r = tf.sqrt(total_degree/normalizer)
    BH_op = (tf.square(r)-1)*tf.diag(tf.ones(shape=[communities*groupsize], dtype=tf.float64))-r*adj+diag 
    val, vec = tf.self_adjoint_eig(BH_op) #this is already normalized so no need to normalize
    subspace = tf.slice(vec, [0,0], [communities*groupsize, dim_proj])
    return r, subspace

def proj_magnitude(space, vector):
    projection_op = tf.matmul(space, tf.transpose(space))
    projection = tf.matmul(projection_op, vector)
    return tf.sqrt(tf.reduce_sum(tf.square(projection))) #tf.reduce_sum(tf.abs(projection))#


def rnd_vec_normed(communities, groupsize, seed=None):
    rnd_vec1 = tf.Variable(tf.random_normal(shape=[communities*groupsize,1], mean=0.0,stddev=1.0,
                                                    dtype=tf.float64,
                                                    seed=seed))
    return normalize_vec(rnd_vec1)


In [11]:
def test_svm_cluster(communities = 2, group_size = 10, seed=1, seed_r=1, p=0.8, q=0.05, name='test1', projection_dim=2, iterations=100, 
                     print_ratio=10, l_rate=0.1, mean=2.0, sd=0.4):
    """testing to see if the loss will decrease backproping through very simple function"""
    B = np.asarray(balanced_stochastic_blockmodel(communities, group_size, p, q, seed)).astype(np.double)
    B = tf.cast(B, dtype = tf.float64)
    
    Diag = tf.diag(tf.reduce_sum(B,0))
    Diag = tf.cast(Diag, tf.float64)

    r =  tf.Variable(tf.random_normal(shape=[1], mean=mean,
                                 stddev=sd, dtype=tf.float64,
                                 seed=seed_r, name=None))

    
    BH = (tf.square(r)-1)*tf.diag(tf.ones(shape=[communities*group_size], dtype=tf.float64))-tf.mul(r, B)+Diag 
    

    with tf.Session() as sess:
        g = tf.get_default_graph()
        
        with g.gradient_override_map({'SelfAdjointEigV2': name}):
            eigenval, eigenvec = tf.self_adjoint_eig(BH)
            #we try to do svm in this subspace 
            #or we can project it down to 1 dimensions, do the clustering there via some threshold and check if it makes sense 
            #by computing the loss, if it is too big, we change the angle we project down to...
            
            
            eigenvec_proj = tf.slice(eigenvec, [0,0], [communities*group_size, projection_dim])
            
            
            
            true_assignment_a = tf.concat(0, [-1*tf.ones([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
            true_assignment_b = -1*true_assignment_a
            true_assignment_a = tf.expand_dims(true_assignment_a, 1)
            true_assignment_b = tf.expand_dims(true_assignment_b, 1)

            
            projected_a = tf.matmul(tf.matmul(eigenvec_proj, tf.transpose(eigenvec_proj)), true_assignment_a)#tf.transpose(true_assignment_a))
            projected_b = tf.matmul(tf.matmul(eigenvec_proj, tf.transpose(eigenvec_proj)), true_assignment_b)#tf.transpose(true_assignment_b))
            
            
            
            loss = tf.minimum(tf.reduce_sum(tf.square(tf.sub(projected_a, true_assignment_a))),
                              tf.reduce_sum(tf.square(tf.sub(projected_b, true_assignment_b))))
            
            optimizer = tf.train.GradientDescentOptimizer(l_rate)
            
            train = optimizer.minimize(loss, var_list=[r])

            eigenvec_grad = tf.gradients(eigenvec, r)
            loss_grad = tf.gradients(loss, r)
            
            
            
            r_op, target = target_subspace(adj=B, groupsize=group_size, communities=communities, diag=Diag, dim_proj=projection_dim)  
            
            r_op_projection_a = tf.matmul(tf.matmul(target, tf.transpose(target)), true_assignment_a)
            r_op_projection_b = tf.matmul(tf.matmul(target, tf.transpose(target)), true_assignment_b)
            r_op_loss = tf.minimum(tf.reduce_sum(tf.square(tf.sub(r_op_projection_a, true_assignment_a))),
                              tf.reduce_sum(tf.square(tf.sub(r_op_projection_b, true_assignment_b))))
            
            init = tf.initialize_all_variables()
            
            
            sess.run(init)
            a,b,c,d= sess.run([r, r_op, r_op_loss, tf.transpose(r_op_projection_a)])
            a_lst = []
            b_lst = []
            c_lst = []
            d_lst = []
            
            a_lst.append(a)
            b_lst.append(b)
            c_lst.append(c)
            d_lst.append(d)
            
            print "initial r: {}. r_op = sqrt(average degree) : {} . Loss associated with r_op: {}. r_op assignments {}.".format(a, b, c, d)
            for i in range(iterations):   
                try: sess.run(train)
                except: 
                    pass
                
                if i%print_ratio==0:  
                    #print i
                    try:
                        a,b,c,d = sess.run([r, loss, tf.gradients(loss, r), tf.transpose(projected_a)]) 
                        a_lst.append(a)
                        b_lst.append(b)
                        c_lst.append(c)
                        d_lst.append(d)
                    except:
                        a,b,c,d = 0, 0, 0, 0 
                        a_lst.append(a)
                        b_lst.append(b)
                        c_lst.append(c)
                        d_lst.append(d)
                    #print "current r: {}, current loss: {}, gradient of loss/r is {} and current assignments (up to sign) {}.".format(a,b,c,d)  

    d = {"r_value": a_lst, "loss": b_lst, "gradient_loss_r": c_lst, "projection": d_lst}
    d = pd.DataFrame(d)
    d.to_csv("/Users/xiangli/Desktop/clusternet/plot_data/r{}rate{}p{}q{}iterations{}step{}.csv".format(mean, l_rate, p, q, iterations, print_ratio))
    return  d
                

                
                

In [12]:
data = test_svm_cluster(name='gradient_no_unitary_adjustment', l_rate=0.0001, seed_r=1, seed = 1, print_ratio=50, iterations=1000)


initial r: [ 2.09713102]. r_op = sqrt(average degree) : 1.96214168703 . Loss associated with r_op: 0.239352945504. r_op assignments [[-0.81008947 -0.89831384 -1.06456991 -1.0936426  -1.05592124 -0.92168635
  -0.88318609 -1.01724697 -1.0936426  -1.05086035  0.98362854  0.98397103
   1.03054178  0.80692791  1.13586066  0.80914196  1.03176237  1.10198787
   1.13586066  0.85180488]].


In [10]:
test_svm_cluster(name='grassman_with_2d', l_rate=0.0001, seed_r=1, seed = 1, print_ratio=5000, iterations=10001)

initial r: [ 2.09713102]. r_op = sqrt(average degree) : 1.96214168703 . Loss associated with r_op: 0.239352945504. r_op assignments [[-0.81008947 -0.89831384 -1.06456991 -1.0936426  -1.05592124 -0.92168635
  -0.88318609 -1.01724697 -1.0936426  -1.05086035  0.98362854  0.98397103
   1.03054178  0.80692791  1.13586066  0.80914196  1.03176237  1.10198787
   1.13586066  0.85180488]].


,gradient_loss_r,loss,projection,r_value
0,0.239353,1.962142,"[[-0.810089472142, -0.89831383716, -1.06456991...",[2.09713101839]
1,[[-970.777875618]],0.247152,"[[-0.808987655317, -0.903750927845, -1.0637507...",[2.18420639851]
2,[[-3.47501116044e-12]],0.324255,"[[-0.801222878438, -0.933206056034, -1.0566055...",[6.15681994387]
3,[[-3.47501116044e-12]],0.324255,"[[-0.801222878438, -0.933206056034, -1.0566055...",[6.15681994387]


In [49]:
test_svm_cluster(name='SelfAdjointEigV2', l_rate=0.0001, seed_r=1, seed = 1, print_ratio=5000, iterations=10001)

initial r: [ 2.09713102]. r_op = sqrt(average degree) : 1.96214168703 . Loss associated with r_op: 0.239352945504. r_op assignments [[-0.81008947 -0.89831384 -1.06456991 -1.0936426  -1.05592124 -0.92168635
  -0.88318609 -1.01724697 -1.0936426  -1.05086035  0.98362854  0.98397103
   1.03054178  0.80692791  1.13586066  0.80914196  1.03176237  1.10198787
   1.13586066  0.85180488]].
0
current r: [ 2.09713102], current loss: 0.244082145495, gradient of loss/r is [array([ 0.03527779])] and current assignments (up to sign) [[-0.80940175 -0.9017661  -1.06406728 -1.09702595 -1.05503979 -0.91305556
  -0.88571072 -1.0136581  -1.09702595 -1.05125222  0.97767908  0.97803705
   1.02975475  0.80666566  1.14222167  0.80900335  1.02415991  1.10372799
   1.14222167  0.85444331]].
5000
current r: [ 2.09712749], current loss: 0.244082021042, gradient of loss/r is [array([ 0.03527778])] and current assignments (up to sign) [[-0.80940177 -0.90176601 -1.0640673  -1.09702587 -1.05503982 -0.91305577
  -0.8857

In [54]:
test_svm_cluster(name='SelfAdjointEigV2', l_rate=0.01, seed_r=1, seed = 1, print_ratio=5000, iterations=10001, mean = 0)

initial r: [ 0.09713102]. r_op = sqrt(average degree) : 1.96214168703 . Loss associated with r_op: 0.239352945504. r_op assignments [[-0.81008947 -0.89831384 -1.06456991 -1.0936426  -1.05592124 -0.92168635
  -0.88318609 -1.01724697 -1.0936426  -1.05086035  0.98362854  0.98397103
   1.03054178  0.80692791  1.13586066  0.80914196  1.03176237  1.10198787
   1.13586066  0.85180488]].
0
current r: [ 0.09713102], current loss: 13.4317194769, gradient of loss/r is [array([-35.38462474])] and current assignments (up to sign) [[-0.0704722  -0.00936025 -0.0777787  -0.05508386 -0.07880056 -1.32332547
  -0.04820161 -0.0327515  -0.05508386 -0.02054714  1.12260788  1.10486448
   0.19324849  0.1558531   0.14156616  0.15051068  1.45895059  0.19786074
   0.14156616  0.12984709]].
5000
current r: [ 0.45097726], current loss: 1.35213560743, gradient of loss/r is [array([ nan])] and current assignments (up to sign) [[-0.76517874 -0.64865546 -0.97596476 -0.84888474 -0.98337332 -1.55159973
  -0.70128906 -1.

InvalidArgumentError: Self Adjoint Eigen decomposition was not successful. The input might not be valid.
	 [[Node: SelfAdjointEigV2_70 = SelfAdjointEigV2[T=DT_DOUBLE, _gradient_op_type="SelfAdjointEigV2", compute_v=true, _device="/job:localhost/replica:0/task:0/cpu:0"](add_70)]]
Caused by op u'SelfAdjointEigV2_70', defined at:
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/xia

In [51]:
test_svm_cluster(name='SelfAdjointEigV2', l_rate=0.0001, seed_r=100, seed =100, print_ratio=5000, iterations=10001)

initial r: [ 2.20383543]. r_op = sqrt(average degree) : 1.97484176581 . Loss associated with r_op: 0.662781930024. r_op assignments [[-0.84520184 -1.04351574 -1.09783797 -1.0543809  -1.01460308 -0.94833992
  -0.83218621 -0.8432221  -0.98416898 -1.00625452  1.0952479   0.95841708
   1.12949094  0.9512771   0.32414146  1.14211636  1.0952479   0.92739904
   0.87003141  1.17413761]].
0
current r: [ 2.20383543], current loss: 0.666961367887, gradient of loss/r is [array([ 0.02081512])] and current assignments (up to sign) [[-0.84855647 -1.05652956 -1.09976991 -1.04980963 -1.00161017 -0.93801109
  -0.84630963 -0.839108   -0.97165024 -1.01364658  1.09876804  0.9595762
   1.12354594  0.93764335  0.32311382  1.15104672  1.09876804  0.928076
   0.87132582  1.17617342]].
5000
current r: [ 2.20383335], current loss: 0.66696132456, gradient of loss/r is [array([ 0.02081508])] and current assignments (up to sign) [[-0.84855645 -1.05652945 -1.09976989 -1.04980967 -1.00161027 -0.93801117
  -0.84630952

In [52]:
test_svm_cluster(name='grassman_with_2d', l_rate=0.0001, seed_r=1000, seed = 100, print_ratio=5000, iterations=10001)

initial r: [ 2.1347376]. r_op = sqrt(average degree) : 1.97484176581 . Loss associated with r_op: 0.662781930024. r_op assignments [[-0.84520184 -1.04351574 -1.09783797 -1.0543809  -1.01460308 -0.94833992
  -0.83218621 -0.8432221  -0.98416898 -1.00625452  1.0952479   0.95841708
   1.12949094  0.9512771   0.32414146  1.14211636  1.0952479   0.92739904
   0.87003141  1.17413761]].
0
current r: [ 2.1347376], current loss: 0.665565074398, gradient of loss/r is [array([-1612.51172712])] and current assignments (up to sign) [[-0.84765907 -1.0529035  -1.09927242 -1.05112167 -1.00523673 -0.94090613
  -0.84234876 -0.84028683 -0.97514447 -1.01161039  1.09780201  0.95925908
   1.12521453  0.94142082  0.32339089  1.14855368  1.09780201  0.9279009
   0.87097998  1.17562105]].
5000
current r: [ 2.29598877], current loss: 0.668938341386, gradient of loss/r is [array([-1929.84889544])] and current assignments (up to sign) [[-0.84962699 -1.06101813 -1.1003416  -1.04814432 -0.99711386 -0.93440902
  -0.8

In [57]:
test_svm_cluster(name='grassman_with_2d', l_rate=0.01, seed_r=1000, seed = 100, mean=-2.0, print_ratio=5000, iterations=10001)

initial r: [-1.8652624]. r_op = sqrt(average degree) : 1.97484176581 . Loss associated with r_op: 0.662781930024. r_op assignments [[-0.84520184 -1.04351574 -1.09783797 -1.0543809  -1.01460308 -0.94833992
  -0.83218621 -0.8432221  -0.98416898 -1.00625452  1.0952479   0.95841708
   1.12949094  0.9512771   0.32414146  1.14211636  1.0952479   0.92739904
   0.87003141  1.17413761]].
0
current r: [-1.8652624], current loss: 19.9780034437, gradient of loss/r is [array([-135.54116955])] and current assignments (up to sign) [[  7.62607883e-03  -1.51394537e-02   8.81541803e-03   2.86532468e-03
    3.67478703e-03   1.75515683e-02  -3.10144305e-05  -2.96726668e-02
    1.84014076e-02  -2.50765873e-02  -3.51752344e-02   8.10403541e-03
    5.12098757e-02  -7.82659851e-02   6.90828779e-02  -2.96589109e-03
   -3.51752344e-02   4.32792318e-02  -3.16514231e-02   2.25691657e-02]].
5000
current r: [-0.50985073], current loss: 19.8773870427, gradient of loss/r is [array([-676.23377394])] and current assign

In [59]:
test_svm_cluster(name='gradient_no_unitary_adjustment', l_rate=0.01, seed_r=1000, seed = 100, mean=-2.0, print_ratio=5000, iterations=10001)

initial r: [-1.8652624]. r_op = sqrt(average degree) : 1.97484176581 . Loss associated with r_op: 0.662781930024. r_op assignments [[-0.84520184 -1.04351574 -1.09783797 -1.0543809  -1.01460308 -0.94833992
  -0.83218621 -0.8432221  -0.98416898 -1.00625452  1.0952479   0.95841708
   1.12949094  0.9512771   0.32414146  1.14211636  1.0952479   0.92739904
   0.87003141  1.17413761]].
0
current r: [-1.8652624], current loss: 19.9780034437, gradient of loss/r is [array([-135.54116955])] and current assignments (up to sign) [[  7.62607883e-03  -1.51394537e-02   8.81541803e-03   2.86532468e-03
    3.67478703e-03   1.75515683e-02  -3.10144305e-05  -2.96726668e-02
    1.84014076e-02  -2.50765873e-02  -3.51752344e-02   8.10403541e-03
    5.12098757e-02  -7.82659851e-02   6.90828779e-02  -2.96589109e-03
   -3.51752344e-02   4.32792318e-02  -3.16514231e-02   2.25691657e-02]].
5000
current r: [-0.50985073], current loss: 19.8773870427, gradient of loss/r is [array([-676.23377394])] and current assign

In [67]:
test_svm_cluster(name='gradient_no_unitary_adjustment', l_rate=0.0001, seed_r=1000, seed = 100, mean=0.0, print_ratio=10000, iterations=400001)

initial r: [ 0.1347376]. r_op = sqrt(average degree) : 1.97484176581 . Loss associated with r_op: 0.662781930024. r_op assignments [[-0.84520184 -1.04351574 -1.09783797 -1.0543809  -1.01460308 -0.94833992
  -0.83218621 -0.8432221  -0.98416898 -1.00625452  1.0952479   0.95841708
   1.12949094  0.9512771   0.32414146  1.14211636  1.0952479   0.92739904
   0.87003141  1.17413761]].
0
current r: [ 0.1347376], current loss: 12.4597011854, gradient of loss/r is [array([-14743.81304199])] and current assignments (up to sign) [[-0.1036862  -0.14970453 -0.24773421 -0.31349935 -1.48332304 -0.59194844
  -0.11930684 -0.28088319 -1.46729067 -0.18289341  0.06520975  0.14465215
   0.26570022  1.53480505  0.08151308  0.10921135  0.06520975  0.1305858
   0.04960254  0.15353925]].
10000
current r: [ 1.60911887], current loss: 0.661699232372, gradient of loss/r is [array([-644.48109844])] and current assignments (up to sign) [[-0.83653327 -1.01485487 -1.09218053 -1.06311159 -1.04301477 -0.9704901
  -0.80

In [14]:
r_list = [i for i in range(-6, 7, 2)]
p_q_pairs = [(0.25, 0.05), (0.25, 0.1),(0.25, 0.15), (0.4, 0.05), (0.4,0.2), (0.4, 0.3), (0.9, 0.05), (0.9, 0.45), (0.9, 0.7)]
l_rate_lst = [10**(-i)/3 for i in range(1, 6, 1)]


l_rate_dic = {i: l_rate_lst[i] for i in range(len(l_rate_lst))}
r_list_dic = {i: r_list[i] for i in range(len(r_list))}
pq_pairs_dict = {i: p_q_pairs[i] for i in range(len(p_q_pairs))}


In [48]:
for l in l_rate_lst:
    {"l_rate_{}".format(l): {k: test_svm_cluster(communities = 2, group_size = 10, seed=1, seed_r=1, p=p_q_pairs[0][0],
                               q=p_q_pairs[0][1], name='grassman_with_2d', projection_dim=1, iterations=20, 
                     print_ratio=10, l_rate=l, mean=k, sd=0.4) for k in r_list}}

initial r: [-5.90286898]. r_op = sqrt(average degree) : 1.22474487139 . Loss associated with r_op: 19.9929257335. r_op assignments [[ -2.13359668e-02  -2.12252378e-02  -4.40538649e-17  -2.10811278e-02
   -2.20158447e-02  -1.95650001e-02  -2.00643753e-02  -1.93911305e-02
   -2.18480916e-02  -1.88223516e-02  -1.86831659e-02  -1.65289835e-02
   -1.79502095e-02  -1.92695692e-02  -2.18605320e-02  -2.00779270e-02
   -1.89744424e-02  -1.37863730e-02  -1.82698821e-02  -1.28737750e-02]].
initial r: [-3.90286898]. r_op = sqrt(average degree) : 1.22474487139 . Loss associated with r_op: 19.9929257335. r_op assignments [[ -2.13359668e-02  -2.12252378e-02  -4.40538649e-17  -2.10811278e-02
   -2.20158447e-02  -1.95650001e-02  -2.00643753e-02  -1.93911305e-02
   -2.18480916e-02  -1.88223516e-02  -1.86831659e-02  -1.65289835e-02
   -1.79502095e-02  -1.92695692e-02  -2.18605320e-02  -2.00779270e-02
   -1.89744424e-02  -1.37863730e-02  -1.82698821e-02  -1.28737750e-02]].
initial r: [-1.90286898]. r_op =

KeyboardInterrupt: 

In [ ]:
for l in range(len(l_rate_lst)):
    for k in range(len(r_list)):
        for j in range(len(p_q_pairs)):
            test_svm_cluster(communities = 2, group_size = 10, seed=2000, seed_r=2000, p=p_q_pairs[j][0],
                               q=p_q_pairs[j][1], name='grassman_with_2d', projection_dim=2, iterations=50000, 
                     print_ratio=1000, l_rate=l_rate_lst[l], mean=r_list[k], sd=0.2)

initial r: [-5.91966987]. r_op = sqrt(average degree) : 0.974679434481 . Loss associated with r_op: 16.0. r_op assignments [[ -1.00000000e+00   0.00000000e+00   7.65495750e-16  -1.00000000e+00
    1.70251820e-15  -1.00000000e+00  -1.82229429e-16  -1.34802380e-16
   -1.00000000e+00   1.24453021e-15   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]].
initial r: [-5.91966987]. r_op = sqrt(average degree) : 1.11803398875 . Loss associated with r_op: 17.415962993. r_op assignments [[ 0.11659691  0.18026741  0.19123962 -1.          0.16254117 -1.
   0.15197424  0.18316447  0.13830385  0.19280159  0.18473155  0.19062751
   0.19513961  0.18352786  0.19711615  0.1958737   0.20082423  0.1912625
   0.1711578   0.19066534]].
initial r: [-5.91966987]. r_op = sqrt(average degree) : 1.2449899598 . Loss associated with r_op: 19.1017150954. r_op assignments [[-0.07803397  0.00585

In [18]:
for l in range(len(l_rate_lst)):
    for k in range(len(r_list)):
        for j in range(len(p_q_pairs)):
            print l_rate_lst[l], r_list[k], p_q_pairs[j]

0.0333333333333 -6 (0.25, 0.05)
0.0333333333333 -6 (0.25, 0.1)
0.0333333333333 -6 (0.25, 0.15)
0.0333333333333 -6 (0.4, 0.05)
0.0333333333333 -6 (0.4, 0.2)
0.0333333333333 -6 (0.4, 0.3)
0.0333333333333 -6 (0.9, 0.05)
0.0333333333333 -6 (0.9, 0.45)
0.0333333333333 -6 (0.9, 0.7)
0.0333333333333 -4 (0.25, 0.05)
0.0333333333333 -4 (0.25, 0.1)
0.0333333333333 -4 (0.25, 0.15)
0.0333333333333 -4 (0.4, 0.05)
0.0333333333333 -4 (0.4, 0.2)
0.0333333333333 -4 (0.4, 0.3)
0.0333333333333 -4 (0.9, 0.05)
0.0333333333333 -4 (0.9, 0.45)
0.0333333333333 -4 (0.9, 0.7)
0.0333333333333 -2 (0.25, 0.05)
0.0333333333333 -2 (0.25, 0.1)
0.0333333333333 -2 (0.25, 0.15)
0.0333333333333 -2 (0.4, 0.05)
0.0333333333333 -2 (0.4, 0.2)
0.0333333333333 -2 (0.4, 0.3)
0.0333333333333 -2 (0.9, 0.05)
0.0333333333333 -2 (0.9, 0.45)
0.0333333333333 -2 (0.9, 0.7)
0.0333333333333 0 (0.25, 0.05)
0.0333333333333 0 (0.25, 0.1)
0.0333333333333 0 (0.25, 0.15)
0.0333333333333 0 (0.4, 0.05)
0.0333333333333 0 (0.4, 0.2)
0.0333333333333

In [17]:
l_rate_lst

[0.03333333333333333,
 0.0033333333333333335,
 0.0003333333333333333,
 3.3333333333333335e-05,
 3.3333333333333337e-06]